In [1]:
%config IPCompleter.greedy=True

In [63]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import math

In [3]:
root_path = './'

## SF Covid-19 Dataset

In [4]:
cases_df = pd.read_csv(root_path + '/data/COVID-19_Cases_Summarized_by_Date__Transmission_and_Case_Disposition.csv')

In [5]:
cases_df.isna().sum()

Date                     0
Transmission Category    0
Case Disposition         0
Case Count               0
dtype: int64

#### We can verify in above query, 0 MISSING values were found 

## Pre-processing steps for SF Covid-19 Dataset

1.   Converting Date column into date-time dtype



In [6]:
cases_df["Date"] = pd.to_datetime(cases_df["Date"], format="%Y/%m/%d")

In [7]:
# Verify by uncommenting and toggling with ascending=True/False
# cases_df.sort_values(by="Date", ascending=False)

## Crime Dataset

In [83]:
crime_df = pd.read_csv(root_path + '/data/Police_Department_Incident_Reports__2018_to_Present.csv')

In [84]:
crime_df["Police District"].value_counts()

Central       53323
Northern      46002
Mission       45971
Southern      42958
Tenderloin    33563
Bayview       27926
Ingleside     24320
Taraval       23231
Richmond      20263
Park          16076
Out of SF      9303
Name: Police District, dtype: int64

In [9]:
crime_df.shape

(342936, 36)

In [10]:
crime_df.isna().sum()

Incident Datetime                                            0
Incident Date                                                0
Incident Time                                                0
Incident Year                                                0
Incident Day of Week                                         0
Report Datetime                                              0
Row ID                                                       0
Incident ID                                                  0
Incident Number                                              0
CAD Number                                               79225
Report Type Code                                             0
Report Type Description                                      0
Filed Online                                            269212
Incident Code                                                0
Incident Category                                          162
Incident Subcategory                                   

## Pre-processing steps for Crime Dataset:


### Considering only non-null columns and Incident columns

In [11]:
columns_to_consider = ["Incident Datetime", 
                       "Incident Date", 
                       "Incident Time", 
                       "Incident Year", 
                       "Incident Day of Week", 
                       "Report Datetime", 
                       "Row ID", 
                       "Incident ID", 
                       "Incident Number", 
                       "Report Type Code", 
                       "Report Type Description", 
                       "Incident Code", 
                       "Incident Category", 
                       "Incident Subcategory", 
                       "Incident Description", 
                       "Resolution", 
                       "Police District"]

In [12]:
crime_df = crime_df[columns_to_consider]

In [13]:
crime_df = crime_df[crime_df["Incident Category"].notna()]

In [14]:
# Verify whhether nulls are removed through the above query
# crime_df.isnull().sum()

In [15]:
crime_df["Incident Date"] = pd.to_datetime(crime_df["Incident Date"], format="%Y/%m/%d")

In [16]:
crime_df["Incident Datetime"] = pd.to_datetime(crime_df["Incident Datetime"], format="%Y/%m/%d %I:%M:%S %p")

In [17]:
crime_df.dtypes

Incident Datetime          datetime64[ns]
Incident Date              datetime64[ns]
Incident Time                      object
Incident Year                       int64
Incident Day of Week               object
Report Datetime                    object
Row ID                              int64
Incident ID                         int64
Incident Number                     int64
Report Type Code                   object
Report Type Description            object
Incident Code                       int64
Incident Category                  object
Incident Subcategory               object
Incident Description               object
Resolution                         object
Police District                    object
dtype: object

## Visualizations

In [18]:
crime_df.head(1)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Police District
0,2020-02-03 14:45:00,2020-02-03,14:45,2020,Monday,2020/02/03 05:50:00 PM,89881675000,898816,200085557,II,Initial,75000,Missing Person,Missing Person,Found Person,Open or Active,Taraval


In [19]:
def convert_agg_df_into_flatten_df(df):
  df = df.reset_index()
  df.columns = [' '.join(col).strip() for col in df.columns.values]

  return df

# df = crime_df.groupby(['Incident Date'])['Incident Date'].agg(["count"]).reset_index()
# df.head()

#### Unique first Reports below are basically unique crimes that have been occured (we filtered according to the below criterias because we want to avoid counting same crime twice with other report types i.e. supplemental reports)

In [20]:
unique_first_reports = crime_df[(crime_df["Report Type Description"] == 'Initial') |
                         (crime_df["Report Type Description"] == 'Initial Supplement') | 
                         (crime_df["Report Type Description"] == 'Vehicle Initial') |
                         (crime_df["Report Type Description"] == 'Coplogic Initial')]

unique_first_reports["Report Type Description"].value_counts()

Initial               213832
Coplogic Initial       65508
Initial Supplement     27882
Vehicle Initial        16277
Name: Report Type Description, dtype: int64

In [21]:
df = unique_first_reports['Incident Date'].value_counts().to_frame().reset_index()
df = df.rename(columns={"index": "date", "Incident Date": "count"})
df = df.sort_values(by=["date"])
df.head()

,date,count
14,2018-01-01,478
453,2018-01-02,380
265,2018-01-03,405
107,2018-01-04,433
156,2018-01-05,424


# Required Inference 2

In [22]:
from datetime import timedelta

## Wald's Test
### One sample test
#### Deaths
Hypotheses:  
$H_0: \hat\mu = \mu_0$  
$H_1: \hat\mu \neq \mu_0$

In [23]:
death_cases = cases_df[cases_df['Case Disposition'] == 'Death']
death_counts = death_cases.groupby('Date')['Case Count'].sum().reset_index()

In [24]:
latest_date = death_counts.sort_values(by='Date', ascending=False)['Date'].iloc[0]

In [25]:
last_week_date = latest_date - timedelta(weeks=1)
second_last_week_date = last_week_date - timedelta(weeks=1)

In [26]:
last_week_death_counts = death_counts[death_counts['Date'] > last_week_date]
last_week_death_counts

,Date,Case Count
19,2020-04-25,1
20,2020-04-28,1
21,2020-04-29,2
22,2020-04-30,1


In [27]:
second_last_week_death_counts = death_counts.loc[(death_counts['Date'] > second_last_week_date) & (death_counts['Date'] <= last_week_date)]
second_last_week_death_counts

,Date,Case Count
17,2020-04-19,1
18,2020-04-22,1


In [28]:
n = 7
second_last_week_mean_deaths = second_last_week_death_counts['Case Count'].sum() / n
lambda_hat = n / last_week_death_counts['Case Count'].sum()

mu_hat = 1 / lambda_hat
mu_0 = second_last_week_mean_deaths
se_hat = mu_hat / np.sqrt(n)

In [29]:
w = np.abs((mu_hat - mu_0) / se_hat)

In [30]:
w

1.5874507866387546

Since the Wald's statistic $|w| \leq Z_{\alpha/2}$ we fail to reject the null hypothesis $H_0$ and conclude that $\hat\mu = \mu_0$

## Z-test
#### Deaths
Hypotheses:  
$H_0: \overline{X} = \mu_0$  
$H_1: \overline{X} \neq \mu_0$

##### Regardless of the original distribution of X, we know by Central Limit Theorem, that $\overline{X} \sim N(\mu, \dfrac{\sigma^2}{n})$
##### where $\mu$ is true mean and $\sigma^2$ is true standard deviation

##### Hence, Z-test is applicable

In [69]:
death_cases = cases_df[cases_df['Case Disposition'] == 'Death']
death_counts = death_cases.groupby('Date')['Case Count'].sum().reset_index()

In [70]:
latest_date = death_counts.sort_values(by='Date', ascending=False)['Date'].iloc[0]
last_week_date = latest_date - timedelta(weeks=1)
second_last_week_date = last_week_date - timedelta(weeks=1)

last_week_death_counts = death_counts[death_counts['Date'] > last_week_date]
second_last_week_death_counts = death_counts.loc[(death_counts['Date'] > second_last_week_date) & (death_counts['Date'] <= last_week_date)]


X = death_counts["Case Count"].astype(float).to_numpy()
e_of_x = np.mean(X)
e_of_x_square = np.mean(np.multiply(X, X))

true_standard_deviation_of_X = math.sqrt(e_of_x_square - (e_of_x**2))

In [72]:
X_bar = last_week_death_counts['Case Count'].mean()
X_bar
# z_statistic = ()

1.25

In [75]:
crime_df["Incident Category"].value_counts()

Larceny Theft                                   109388
Other Miscellaneous                              25695
Non-Criminal                                     21192
Malicious Mischief                               20748
Assault                                          20298
Burglary                                         15412
Motor Vehicle Theft                              12658
Warrant                                          12338
Lost Property                                    12292
Fraud                                            10106
Recovered Vehicle                                10102
Drug Offense                                      8637
Robbery                                           8151
Missing Person                                    7834
Offences Against The Family And Children          6646
Suspicious Occ                                    6418
Disorderly Conduct                                6049
Traffic Violation Arrest                          4384
Miscellane

0.6330530338504798

In [78]:
crime_df["Incident Subcategory"].value_counts()

Larceny - From Vehicle                        62830
Other                                         46566
Larceny Theft - Other                         26591
Vandalism                                     20325
Simple Assault                                13132
                                              ...  
Robbery - Residential                            15
Bribery                                           5
Arrest                                            3
Human Trafficking, Involuntary Servitude          1
Human Trafficking (A), Commercial Sex Acts        1
Name: Incident Subcategory, Length: 75, dtype: int64

In [80]:
crime_df["Incident Description"].value_counts()

Theft, From Locked Vehicle, >$950                             46697
Lost Property                                                 12292
Theft, Other Property, $50-$200                               10769
Malicious Mischief, Vandalism to Property                     10291
Battery                                                       10116
                                                              ...  
Abortion, Unlawful                                                1
Burglary, Safe,  Warehouse                                        1
Fighting Dog or Animal, Biting or Causing  Injury or Death        1
Vehicle Alarm Code Grabbing Device, Possess or Use                1
Threat to State Official or Judge                                 1
Name: Incident Description, Length: 785, dtype: int64

In [81]:
crime_df.isna().sum()

Incident Datetime          0
Incident Date              0
Incident Time              0
Incident Year              0
Incident Day of Week       0
Report Datetime            0
Row ID                     0
Incident ID                0
Incident Number            0
Report Type Code           0
Report Type Description    0
Incident Code              0
Incident Category          0
Incident Subcategory       0
Incident Description       0
Resolution                 0
Police District            0
dtype: int64